<a href="https://colab.research.google.com/github/hduongck/AI-ML-Learning/blob/master/Machine_Learning_Lesson_1%262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lesson 1:

# Syllabus in brief

Depending on time and class interests, we'll cover something like (not necessarily in this order):

Train vs test

- Effective validation set construction

Trees and ensembles

- Creating random forests
- Interpreting random forests

What is ML? Why do we use it?

- What makes a good ML project?
- Structured vs unstructured data
- Examples of failures/mistakes 

Feature engineering

- Domain specific - dates , URLs, text
- Embeddings / latent factors

Regularized models trained with SGD

- GLMs, Elasticnet, etc (NB: see what James covered)

Basic neural nets

- PyTorch
- Broadcasting, Matrix Multiplication
- Training loop, backpropagation

KNN

CV/boostrap(Diabetes data set?)

Ethical considerations

Skip:

- Dimensionality reduction
- Interactions
- Monitoring training
- Collaborative filtering
- Momentum and LR annealing



# Random Forest : Blue Book for Bulldozers



In [0]:
#@title Import Library and getting Data

%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install fastai==0.7.0
!pip install torchtext==0.2.3

!pip install kaggle
!mkdir .kaggle

import os
import json 
token = {"username":"hduongck","key":"983e2ab1fbb29cf2734bcbf8811d42fb"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

os.makedirs('/data/bulldozers/', exist_ok=True)
os.makedirs('/.kaggle/',exist_ok=True) 
!chmod 600 /content/.kaggle/kaggle.json

!kaggle config set -n path -v{/content/data}



!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle competitions download -c bluebook-for-bulldozers -p /content/data/bulldozers


!unzip /content/data/bulldozers/\Train.zip -d data/bulldozers

#!mkdir data/rossmann
#!wget http://files.fast.ai/part2/lesson14/rossmann.tgz
#!tar -xvzf /content/rossmann.tgz -C /content/data/rossmann


In [0]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from IPython.display import display

from sklearn import metrics

# Introduction to Blue Book for Bulldozers

**About..... **

**....our teaching**

At fast.ai we have a distinctive teaching philosophy of the whole game. This is different from how most traditional math & technical courses are taught, where you have to learn all the individual elements before you can combine them (Harvard processor David Perkins call this elementilis) but it is similar to how topics like driving and baseball are taught. That is, you can start driving without knowing how an internal combustion engine works, and children begin playing baseball before they learn all the formal rules.

**..... our approach to machine learning**

Most machine learning courses will throw at you dozens of different algorithms, with a brief technical description of the math behind them and maybe a toy example. You're left confused by the enormous range of techniques shown and have little practical understanding of how to apply them. 

The good news is that modern machine learning can be distilled down to a couple of key techniques that are of very wide applicalbility. Recent studies have shown that the vast majority of datasets can be best modeled with just two methods:

- Ensembles of decision trees (i.e. Random Forests and Gradient Boosting Machines), mainly for structured data (such as you might find in a database table at most companies)
- Multi-layered neural networks learnt with SGD (i.e shallow and/or deep learning), mainly for unstructured data(such as audio, vision and natural language)

In this course we'll be doing a deep dive into random forests, and simple models learnt with SGD. You'll be learning about gradient boosting and deep learning in part 2.

**.. this dataset**

We will be looking at the Blue Book for Bulldozens Kaggle competition: "The goal of the contest is to predict the sale price of a particular piece of heavy equipment at auction based on it's usage, equipment type, and configuration. The data is sourced from auction result posting and includes information on usage and equipment configurations."

**... Kaggle competitions**

Kaggle is an awesome resource for aspiring data scientist or anyone looking to improve their machine learning skills. There is nothing like being able to get hands-on practice and receiving real-time feedback to help you improve your skills.

Kaggle provides : 
1. Interesting datasets
2. Feedback on how you're doing
3. A leader board to see what's good , what's possible and what's state of art
4. Blog posts by winning contestants share useful tips and techniques



# Let's look at the data [25:25](https://youtu.be/CzdWqFTmn0Y?t=25m25s)

For this competition, you are predicting the sale price of bulldozers sold at auctions . The data for this competition is split into three parts:
- **Train.csv** is the training set, which contains data through the end of 2011.
- **Valid.csv** is the validation set, which contains data from Jan 1, 2012 - Apr 30,2012. You make predictions on this set throughout the majortity of the competition. Your score on this this set is used to create the public leaderboard.
- **Test.csv** is the test set, which won't be released until the last week of the competition. It contains data from May 1,2012 - Nov 2012. Your score on the test set determines your final rank for the competition.

The key fields are in train.csv are :

- SaleID : the unique identifier of the sale
- MachineID: the unique identifier of a machine. A machine can be sold multiple times 
- saleprice: what the machine sold for at auction (only provided in train.csv)
- saledate : the date of the sale

Type of data
:
- **Structured data**: Columns representing a wide range of different types of things such as identifiers , currency, data, size. 
- **Unstructured data**: Images

pandas is the most important library when you are working with structured data which is usually imported as pd.



In [0]:
PATH = "data/bulldozers/"

In [0]:
!ls {PATH}

Data%20Dictionary.xlsx		  Train.7z	     Train.zip
Machine_Appendix.csv		  TrainAndValid.7z   Valid.7z
median_benchmark.csv		  TrainAndValid.csv  Valid.csv
random_forest_benchmark_test.csv  TrainAndValid.zip  ValidSolution.csv
Test.csv			  Train.csv	     Valid.zip


In [0]:
df_raw = pd.read_csv(f'{PATH}Train.csv',low_memory=False, 
                     parse_dates=['saledate'])


In [0]:
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- parse_dates -- A list of any columns that contain dates
- low_memory = False -- Forces it to read more of the file to decide what the types are.


In [0]:
def display_all(df):
  with pd.option_context("display.max_rows", 1000):
    with pd.option_context("display.max_columns", 1000):
      display(df)
      
display_all(df_raw.tail().transpose())

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


The variable we want to predict is called Dependent Variable in this case our dependent variable is SalePrice.

**Question**: Should you never look at the data because of the risk of overfit? [33:08](https://youtu.be/CzdWqFTmn0Y?t=33m8s)

We want to find out at least enough to know that we have managed to import okay, but tend not to really study it at all at this point, because we do not want to make too many assumptions about it. Many books say to do a lot exploratory data analysis (EDA ) first. We will learn machine learning driven EDA today.

#Purpose of the project - Evaluation [34:36](https://youtu.be/CzdWqFTmn0Y?t=34m6s)

Root mean squared log error. The reason we use log is because generally, you care not so much about missing by usd10 but missing by 10% .So if it was usd1,000,000 item and you are usd100,000 off or if it was a usd10,000 item and you are usd1000 off - we would consider those equivalent scale issues. 


In [0]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

- np - Numpy lets us treat arrays, matrices, vectors, high dimensional tensors as if they are Python variables.

# What is Random Forest [36.37](https://youtu.be/CzdWqFTmn0Y?t=36m37s)

Random Forest is a universal machine learning technique.

- It can predict something that can be of any kind -- it could be a category( classification), a continuous variable( regression).
- It can predict with columns of any kind -- pixels, zip codes, revenues, etc (i.e both structured and unstructed data).
- It does not generally overfit too badly, and it is very easy to stop it from overfitting.
- You do not need a separate validation set in general. It can tell you how well it generalizes even if you only have one dataset.
- It has few, if any, statistical assumption. It does not assume that your data is normally distributed, the relationship is linear, or you have specified interactions. 
- It requires very few pieces of feature engineering. For many different types of situation, you do not have to take the log of the data or multiply interactions together. 

**Question: What about a curse of dimensionality?** [38:16](https://youtu.be/CzdWqFTmn0Y?t=38m16s)

There are two concepts you often hear -- curse of dimensionality and no free lunch theorem. They are both largely meaningless and basically stupid and yet many people in the field not only know that but think the opposite so it is well worth explaining. 
- **The curse of dimensionality** is this idea that the more columns you have, it creates a space that is more and more empty. There is this fascinating mathematical idea that the more dimensions you have, the more all of the points sit on the edge of the space. If you just have a single dimension where things are random, then they are spread out all over. Where else, if it is a square then the probability that they are in the middle means that they cannot be on the edge of either dimension so it is a little less likely that they are not on the edge. Each dimension you add, it becomes multiplicatively less likely that the point is not on the edge of at least one dimension, so in high dimensions, everything sits on the edge. What tha means in theory is that the distance between points is much less meaningful . So if we assume it matters, then it would suggest that when you have lots of columns and you just use them without being careful to remove the ones you do not care about that things which will not work. This turns out not to be the case for number of reasons.
    - Points still do have different distances away from each other. Just because they are on the edge, they still vary on how far away they are from each other and so this point is more similar at this point than it is to that point. 
    - So things like k-nearest neighbors actually work really well in high dimensions despite what the theoretician claimed. What really happened here was that in 90's , theory took over machine learning. There was this concept of support vector machines(SVM) that was theoretically well justified, extremely easy to analyze mathematically, and you can prove things about them-- we lost a decade of real practical development. And all these theories became very popular like the curse of dimensionality.** Nowadays the world of machine learning has become very empirical and it turns out that in practice, building models on lots columns works really well**
    
- **No free lunch theorem** [41:08]: their claim is that is there no type of model that works well for any kind of dataset. In the mathematical sense, any random dataset by definition is random, so there is not going to be some way of looking at every possible random dataset that is in someway more useful than any other approach. In real world, we look at data which is not random. Mathematically we would say it sits on some lower dimensional manifold. It was created by some kind of causual structure. There are some relationships in there, so the truth is that we are not using random datasets and hence there are actually techniques that work much better than other techniques for nearly all of the datasets you look at. Nowadays, there are empirical researchers who study which techniques work a lot of the time. Ensembles of decisions trees, which random forest for one, is perhaps the technique which most often comes at the top. Fast.ai provides a standard way to pre-process them properly and set their parameters. 



#Scikit-learn [42:54](https://youtu.be/CzdWqFTmn0Y?t=42m54s)

Most popular and important package for machine learning in Python. It is not the best at everything (e.g XGBoost is better than Gradient Boosting Tree), but pretty good at nearly everything.

In [0]:
m = RandomForestRegressor(n_jobs=-1)

- RandomForestRegressor - regressor is a method for predicting continuois variables (i.e regression)
- RandomForestClassifier - classifier is a method for predicting categorical variables (i.e classification)

Alot of people misunderstand the term "Regressor" related to " Linear Regression"

Everything in scikit-learn has the same form
- Create an instance of an object for the machine learning model
- Call fit by passing in the independent variables ( the things you are going to use to predict) and dependent variable
- in df.drop() : axist =1 means remove columns
- shift+tab in Jupyter Notebooks will bring up the inspection of parameters of a function
- "list-like" means anything you can index in Python

In [0]:
m.fit(df_raw.drop('SalePrice',axis=1),df_raw.SalePrice)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: ignored

The above code will result in an error. There was a value inside the dataset " Conventional", and it did not know how to create a model using that String. We have to pass numbers to most machine learning models and certainly to random forests. So step 1 is to convert everything into numbers. 

This dataset contains a mix of **continous** and **categorical** variables
- continuous -> numbers where the meaning is numeric such as price
- categorical -> either numbers where the meaning is not continuous like zip code or string such as "large", "medium", "small"

Here are some of the information we can extract from date - year, month, quarter, day of month, day of week, week of year, it is a holiday? weekend? was it raining? was there a sport event that day? It really depends on what you are doing. If you are predicting soda sales in SoMa, you would probably want to know if there as a San Francisco Giants ball game that day. What is in a date is one of the most important piece of feature engineering you can do and no machine learning algorithm can tell you whether the Giants were playing on that day and that it was important. So this is where you need to do feature engineering. 

The **add_datepart** method extracts particular date fields from a complete datatime for the purpose of constructing categoricals. You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a  function of time of these granularities.


In [0]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, 
                                     infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 
            'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 
            'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)

- getattr -- look inside of an object and finds an attribute with that name.
- drop = True -- unless specified, it will drop the date time field because we cannot use "saledate" directly because it is not a number.

In [0]:
fld = df_raw.saledate
fld.year

AttributeError: ignored

- fld -> Pandas series
- fld.year -> AttributeError: 'Series' object has no attribute 'year' because it only apply to Pandas series that are datetime objects. So what Pandas does is it splits out different methods inside attributes that are specific to what they are. So date time objects will have dt attribute defined and that is where you will find all the date time specific attributes. 

In [0]:
fld.dt.year.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saledate, dtype: int64

Now apply add_datepart to df_raw

In [0]:
add_datepart(df_raw,'saledate')


In [0]:
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

**Question**: [55:40] What is the difference between df['saleYear'] and df.saleYear?

It is safer to use square brackets especially when assigning values and there is a possibility that the column does not already exist.

**After running add_datepart, it added many numercial columns and removed saledate column. This is not quite enough to get passed the error we saw earlier as we still have other columns that contain string values.** Pandas has a concept of a category data type, but by default it would not turn anything into a category for you. Fast.ai provides a function called **train_cats** which creates categorical variables for everything that is a String. Behind the scenes, it creates a column that is an integer and it is going to store a mapping from the integers to the strings.**train_cats is called "train" because it is training data specific**. It is important that validation and test sets will use the same category mappings ( in other words, if you used 1 for "high" for a training dataset, then 1 should also be for "high" in validation and test datasets). **For validation and test dataset, use apply_cats** instead.


In [0]:
train_cats(df_raw)

In [0]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

- **df_raw.Usageband.cat** -> similar to **fld.dt.year**, .**cat** gives you access to things assuming something is a category

The order does not matter too much, but since we are going to be creating a decision tree that split things at a single point(i.e High vs Low and Medium; High and Low vs Medium) which is a little bit weird. To order them in a sensible manner, you can do the following:

In [0]:
df_raw.UsageBand.cat.set_categories(['High','Medium','Low'],ordered=True, inplace=True)

- **inplace** will ask Pandas to change the existing dataframe rather than returning a new one.

There is a kind of categorical variable called "ordinal". An ordinal categorical variable has some kind of order (e.g "Low"<"Medium"<"High"). Random forests are not terribly sensitive for that fact, but it is worth noting.

We're still not quite done- for instance we have lots of missing values, wish we can't pass directly to a random forest.

In [0]:
df_raw.saledate

AttributeError: ignored

In [0]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

The above will add number of empty values for each series, we sort them by the index (pandas.Series.sort_index) and divide by a number of dataset.

Reading CSV took about 10 seconds, and processing took another 10 seconds, so if we do not want to wait again, it is a good idea to save them. Here we will save it in a feather format. What this is going to save it to disk in exactly the same basic format that it is in RAM. This is by far the fastest way to save something, and also to read it back. Feather format is becoming standard in not only Pandas but in Java, Apache Spark, etc.

In [0]:
os.makedirs('tmp',exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

# Pre-processing

In [0]:
import pyarrow.feather as feather

In [0]:
df_raw = feather.read_feather('tmp/bulldozers-raw')

We will replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [0]:
df,y,nas = proc_df(df_raw,"SalePrice")

![alt text](https://cdn-images-1.medium.com/max/600/1*4oMdeYV25o5H2LWIAT01EA.png)

- df -> dataframe
- y_fld - name of dependent variable
- It makes acopy of the dataframe, grab the dependent variable values (y_fld), and drop the dependent variable from the dataframe.
- Then it will fix_missing (see below)
- We then go through the dataframe and call **numericalize** (see below).
- dummies -> there are columns with a small number of possible values, you can put into dummies instead of numericalizing them. But we will not do that for now. 

**fix_missing:**

![alt text](https://cdn-images-1.medium.com/max/600/1*r99PY_3lknL_9_xqUJuYIw.png)

- For numeric data type, first we check if there is null column. If so, it will create a new column with a name with _na appended at the end and set it to 1 if it is missing; 0 otherwise(boolean). It will then **replace the missing value with a median.**
- We do not need to do this for categorical variables because Pandas handles them automatically by setting them to -1

**numericalize**

![alt text](https://cdn-images-1.medium.com/max/750/1*7mCv-uPACBVYTZtzY1dovg.png)

- If it not numeric and is a categorical type, we replace the column with its code plus 1. By default pandas uses -1 for missing, so now missing have an ID of 0.

In [0]:
df.head()

Now we have all numerical values. Note that booleans are treated as numbers. So we can create a random forest.

In [0]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df,y)
m.score(df,y)

- Random forests are **trivially parallelizable** -- meaning if you have more than one CPU, you can split up the data across different CPUs and it linearly scale. So the more CPUs you have, it will divide the time it takes by that number (not exactly but roughly). **n_job=-1** tells the random forest regressor to create a separate job/process for each CPU you have.
- m.score will $r^2$ value (1 is good , 0 is bad). We will learn $r^2$ next week.
- Wow, an $r^2$ of 0.98 -> that's great, right? Well, perhap not....

Possibly the most important idea in machine learning is that of having separate training and validation data sets. As motivation,  suppose you don't divide up your data, but instead use all of it. And suppose you have lots of parameters:

![alt text](https://cdn-images-1.medium.com/max/600/1*3O5pvKZ95nzJsSYamsfDLA.png)

The error for the pictured data points is lowest for the model on the far right(the blue curve passes through the red points almost perfectly), yet it's not the best choice. Why is that ? If you were to gather some new data points, they most likely would not be on that curve in the graph on the right, but would be closer to the curve in the middle graph.

This illustrates how using all our data can lead to** overfitting.** A validation set helps diagnose this problem.


In [0]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 1200 #same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw,n_trn)
X_train,X_valid = split_vals(df,n_trn)
y_train,y_valid = split_vals(y,n_trn)

X_train.shape, y_train.shape, X_valid.shape

**Base Model**

By using validation set, you see that the $r^2$ is 0.88 for validation set. 

In [0]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [0]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train,y_train)
print_score(m)

An r^2 in the high-80's isn't bad at all (and the RMSLE puts us around rank 100 of 470 on the Kaggle leaderboard), but we can see from the validation set score that** we're over-fitting badly**. To understand this issue, let's simplify things down to a single small tree.

#Lesson 2: Random Forest Deep Dive 
[Video](https://youtu.be/blyXCk4sgEg)

For the next couple, we will look at : 
- How random forests actually work
- what to do if they do not work properly
- what the pros and cons are
- what we can tune
- How to interpret the result

Fastai library is collections of best techniques to achieve state-of-art result. For structured data analysis, scikit-learn has a lot of great code. So what fast.ai is to help us get things into scikit-learn and then interpret things out from scikit-learn. 

As we noted, it is very important to deeply understand the evaluation metric [06:00](https://youtu.be/blyXCk4sgEg?t=6m)

**Root Mean Squared Logarithmic Error (RMSLE):**

![alt text](https://cdn-images-1.medium.com/max/800/1*aQKT75DvBnh1B6YWKUoJ1w.png)
![alt text](https://cdn-images-1.medium.com/max/800/1*C4EHz1x8aSnzXFDdpUXZIQ.png)

So we took the log of the price and use RMSE


```
df_raw.SalePrice = np.log(df_raw.SalePrice)
```
Then we made everything in the dataset to numbers by doing the following:
- **add_datepart**-> extract date-time features Elapsed represents how many days are elapsed since Jan1st,1970.
- **train_cats** -> converts **string** to pandas **category** data type. We then replace categorical columns with category codes by running **proc_df**
- **proc_df** also replaces missing values of the continuous columns with the median, adds the column called **[Column name]_na** and sets it to true to indicate it was missing.







In [0]:
m = RandomForestRegressor(n_jobs = -1)
m.fit(df, y)
m.score(df, y)

#What is $R^2$?
[10:50](https://youtu.be/blyXCk4sgEg?t=10m50s)

![alt text](https://cdn-images-1.medium.com/max/800/1*tiu0Zw6grKXPvtIYESpRkQ.png)

- $y_i$ : actual/target data
- $\bar{y}$: the average (mean)
- $SS_{tot}$: how much the data vary
- The simplest non-stupid model we came up with last week was create a column of the mean and submit that to Kaggle. In that case, RMSE = $SS_{tot}$ (i.e RMSE of a naive model)
- $f_i$ : predictions 
- $SS_{res}$ is RMSE of the actual model
- If we were exactly as effective as just predicting the mean, $SS_{res}$/$SS_{tot}$ = 1  and $R^2$ = 0
- If we were perfect (i.e $f_i$ = $y_i$ for all cases), $SS_{res}$/$SS_{tot}$ = 0 and $R^2$ = 1

**What is possible range of $R^2$** [14:43](https://youtu.be/blyXCk4sgEg?t=14m43s)

Correct answer : Anything equal to or less than 1. If you predicted infinity for every row, $R^2$ = 1-$\infty$

So when your $R^2$ is negative , it means your model is worse than predicting the mean.

$R^2$ is not necessarily what you are actually trying to optimize, but it is a number you can use for every model and you can start to get a feel of what 0.8 or 0.9 look like. Something you may find interesting is to create synthetic 2D datasets with different amounts of random noise, and see what they look like on a scatterplot and their $R^2$ to get a feel of how close they are to the actual value. 

$R^2$ is the ratio between how good your model is (RMSE) vs how good is the naive mean model (RMSE)

**Overfitting** [17:33](https://youtu.be/blyXCk4sgEg?t=17m33s)

In our case, $R^2$ = 0.98 is a very good model. However, it might be the case that it looks like the one on the right.

![alt text](https://cdn-images-1.medium.com/max/800/1*3O5pvKZ95nzJsSYamsfDLA.png)

It is good at running through the points we gave it (training set) but it is not going to be very good at running through points we did'nt give it (validation set). That is why we always want to have validation set.

Creating a validation set is the most important thing you need to do when you are doing a machine learning project. What you need to do is to come up with a dataset where the score of your model on that dataset is going to be representative of how well your model is going to do in the real world.

If your dataset has a time piece in it (as is in BlueBook competition), you would likely want to predict future prices/values/etc. What Kaggle did was to give us data representing a particular data range in the training set, and then the test set presented a future set of dates that wasn't represented in the training set. So we need to create a validation set that has the same properties:


In [0]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw,n_trn)
X_train,X_valid = split_vals(df,n_trn)
y_train, y_valid = split_vals(y,n_trn)

X_train.shape, y_train.shape, X_valid.shape

We now have something which hopefully looks like Kaggle test set- close enough that using this would give us reasonable accurate score. The reason we want to build a model that is going to work well in the production. 

**Question:** Could you please explain the difference between a validation set and a test set [20:58](https://youtu.be/blyXCk4sgEg?t=20m58s)

One of the things we are going to learn today is how to set hyper parameters.** Hyper parameters** are tuning parameters that awre going to change how your model behaves.If you just have one holdout set(i.e one set of data that you are not using to train with) and we use that to **decide which set of hyper parameter to use.** If we try a thousand different sets of hyper parameters, we may end up overfitting to that holdout set. So what we want to do is to have a second holdout set (the test set) where we can say I have done the best I can and now just once right at the end, I am going to see whether it works. 

You must actually remove the second holdout set (test set) from the data, give it to somebody else, and tell them not let you look at it until you are finished. Otherwise it is so hard not to look at it. In the world of psychology and sociology, it is known as replication crisis or P-hacking. That is why we want to have a test set.

**Question:** We have converted categorical variable into numbers but other models convert it to different columns using one hot encoding - which approach should we use [22:55](https://youtu.be/blyXCk4sgEg?t=22m55s)? We are going to tackle that today.



In [0]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train),y_train), rmse(m.predict(X_valid),y_valid),
          m.score(X_train,y_train),m.score(X_valid,y_valid)]
    if hasattr (m,'oob_score_'): res.append(m.oob_score_)
    print (res)

In [0]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train,y_train)
print_score(m)

$R^2$ is 0.98 on the training set, and only 0.885 on the validation set which makes us think that we are overfitting quite badly. But not too badly as RMSE of 0.25 would have put us in the top 25% of the competition

**Question :** Why not choose random set of rows as a validation set [24:19](https://youtu.be/blyXCk4sgEg?t=24m19s)

Because if we did that , we would not be replicating the test set. If you actually look at the dates in the test set, they are a set of dates what are most recent than any date in the training set. So if we used a validation set that was a random sample, that is much easier because we are predicting the value of a piece of industry equipment on this day when we already have some observations from that day. In general, anytime you are building a model that has a time element, you want your test set to be a separate time period and therefore you really need your validation set to be of separate time period as well.

**Question: **Would't it eventually overfit to the validation set? [25:30](https://youtu.be/blyXCk4sgEg?t=25m30s) 

Yes, actually that is the issue. That would eventually have the possiblity of overfitting on the validation set and when you try it on the test set or submit it to Kaggle, it turns out not to be very good. This happens in Kaggle competitions all the time and they actually have a fourth dataset which is called the private leader board set. Everytime you submit to Kaggle, you actually only get feedback on how well it does on the public leader board set and you do not know which rows they are. At the end of competition, you get judged on a different dataset entirely called private leader board set. The only way to avoid this is to actually be a good machine learning practioner and know how to set these parameters as effectively as possible which we are going to be doing partly today and over the next few weeks.

**[PEP8 standard](https://www.python.org/dev/peps/pep-0008/)** [27:09](https://youtu.be/blyXCk4sgEg?t=27m9s)

`def rmse(x,y): return math.sqrt(((x-y)**2).mean())`

This is one of these examples where the code does not follow PEP8. This should be:

`def rmse(preds,act) : return math.sqrt (((preds-act)**2).mean())`

Being able to look at something in one go with your eyes and overtime learn to immediately see what is going on has a lot of values. Also consistenly use particular letters or abbreviation to mean particular things works well in data science. But if you are doing take-home interview test, follow PEP8 standard.

**Execution time** [29:29](https://youtu.be/blyXCk4sgEg?t=29m29s)

If you put %time, it will tell you how long things took. The rule of thumb is that if something takes more than 10 seconds to run, it is too long to do interactive analysis with it. So what we do is we try to make sure that things can run in a reasonable time. And then when we are finished at the end of the day, we can say ok, this feature engineering, these hyper parameters, etc are all working well, and we will now re-run it the big slow precise way.

One way to speed things up is to pass in the subset parameter to proc_df which will randomly sample the data:


In [0]:
df_trn, y_trn, nas=proc_df(df_raw,'SalePrice',subset=30000,na_dict=nas)
X_train,_ = split_vals(df_trn,20000)
y_train,_ = split_vals(y_trn,20000)

- **subset =30000:** random sample in 30,000 rows
- Be careful to make sure that validation set does not change
- Also make sure that training set does not overlap with the dates.

As you see above, when calling **split_vals**, we do not put the result to a validation set. **_** indicates that we are throwing away the return value. We want to keep validation set the same all the time.

After resampling the training set into the first 20,000 out of a 30,000 subsets, it runs in 621 milliseconds.

#Building a single tree [31:50](https://youtu.be/blyXCk4sgEg?t=31m50s)

We are going to build a forest made of trees. Let's start by looking at trees. In scikit-learn, they do not call them trees but **estimators.**


In [0]:
m = RandomForestRegressor(n_estimators=1,max_depth=3,
                         bootstrap=False,n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

- n_estimators =1 -> create a forest with just one tree
- max_depth = 3 -> to make it a small tree
- bootstrap = False -> random forest randomizes bunch of things, we want to turn that off by this parameter

This small deterministic tree has $R^2$ of 0.393 after fitting so this is not a good model but better than the mean model since it is greater than 1 and we can actually draw [33:00](https://youtu.be/blyXCk4sgEg?t=33m)

In [0]:
draw_tree(m.estimators_[0],df_trn,precision=3)

A tree consists of a sequence of binary decisions.

- The first line indicates the binary split criteria .
- **samples** at the root is 20,000 since that is that we specified when splitting the data. 
- Darker color indicates higher **value**.
- **value** is average of the log of price, and if we built a model where we just used the average all the time , then the mean squared error **mse** would be 0.457
- The best single binary split we can make turns out to be **Coupler_system <= 0.5** which will improve **mse** to 0.115 in false path and only a bit to 0.398 in true path.

We want to build a random forest from scratch [36:28](https://youtu.be/blyXCk4sgEg?t=36m28s). The first step is to create a tree. That creates the first binary decision. How are you going to do it?
- We need to pick a variable and the value to split on such that two groups are as different to each other possible.
- For each variable, for each possible value of that variable see whether it is better?
- How to determine if it better? **Take weighted average of two new nodes.**
- The resulting model will be similar to the naive model of means -> we have a model with a single binary decision. For everybody with coupler_system greater than 0.5, we will fill in 10.211 for everybody else we will put 9.174. Then we will calculate RMSE of this model. 

We know have a single number that represents how good a split is which is the weighted average of the mean squared errors of the two groups that creates [42:13](https://youtu.be/blyXCk4sgEg?t=42m13s). We also have a way to find the best split which is to try every variable and to try every possible value of that variable and see which variable and which value gives us a split with the best score.

Question: Are there circumstances when it is better to split into 3 groups? [45:12](https://youtu.be/blyXCk4sgEg?t=45m12s). It is never necessary to do more than one split at a level because you can just split them again.

This is the entirely of creating a decision tree. Stopping condition:
- When you hit the limit that was requested (**max_depth**)
- When your leaf nodes only have one thing in them.

**Let's make our decision tree better** [46:10](https://youtu.be/blyXCk4sgEg?t=46m10s) 

Right now, our decision tree has $R^2$ of 0.39. Let's make it better by removing max_depth =3. By doing so, the training $R^2$ becomes 1 (as expected since each leaf node contains exactly one element) and validation $R^2$ is 0.58 -- which is better than the shallow tree but not as good as we would like.




In [0]:
m = RandomForestRegressor(n_estimators=1, bootstrap=False,
                         n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

To make these trees better, we will create a forest. To create a forest, we will use a statistical technique called** bagging**

#Bagging [47:12](https://youtu.be/blyXCk4sgEg?t=47m12s)

Michael Jordan developed a technique called the **Bag of Little Bootstraps** in which he shows how to use bagging for absolutely any kind of model to make it more robust and also to give you confidence intervals.

**Random forest -- a way of bagging trees.**

So what is bagging? Bagging is an interesting idea which is what if we created five different models each of which was only somewhat predictive but the models gave predictions that were not correlated with each other. That would mean that the five models would have profound different insights into the relationships in the data. If you took the average of those five models, you are effectively bringing in the insights from each of them. So this idea of averaging models is a technique for **Ensembling**.

What if we created a whole a lot of trees -- big, deep, massively overfit trees but each one, let's say, we only pick a random 1/10 of the data. Let's say we do that hundred times (different random sample each time). They are overfitting terribly but since they are all using different random samples, they all overfit in different ways on different things. In other words, they all have errors but the errors are random. The average of a bunch of random errors is zero. If we take the average of these trees , each of which have been trained on a different random subset, the error will average out to zero and which is left is true relationship -- and that's the random forest.


In [0]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

**n_estimator** by default is 10 (remember, estimators are trees)

**Question: **Are you saying we average 10 crappy models and we get a good model? [51:25] Exactly. Because the crappy models are based on different random subsets and their errors are not correlated with each other. If the errors were correlated, this will not work.

The key insight here is to construct multiple models which are better than nothing and where the errors are, as much as possible, not correlated with each other. 

The number of trees to use is the first of our ** hyperparameters** we are going to tune to achieve higher metric.
 
**Question**: The subset you are selecting, are they exclusive? Can there be overlaps? [52:27](https://youtu.be/blyXCk4sgEg?t=52m27s)

We talked about picking 1/10 at random, but what scikit-learn does by default is for n rows with replacement -- which is called **bootstrapping**. If memory serves correctly, on average, 63.2% of the rows will be represented and many of them will be appear multiple times.

The entire purpose of modeling in machine learning is to find a model which tells you which variables are important and how do they interact together to drive your dependent variable. In practice, the difference between using the random forest space to find your nearest neighbors vs Euclidean space is the difference between a model that makes good predictions and the model that makes meaningless predictions. 

The effective machine learning model is accurate at finding the relationships in the training data and generalizes well to new data [55:53](https://youtu.be/blyXCk4sgEg?t=55m53s). In bagging, that means each of your individual estimators, you want them to be as predictive as possible. The research community found that the more important thing seems to be creating  uncorrelated trees rather than more accurate trees. In scikit-learn, there is another class called **ExtraTreeClassifier** which is an extremely randomized tree model. Rather than trying every split of every variable, it randomly tries a few splits of a few variables which makes training much faster and it can build more trees -- better generalization. If you have crappy individual models, you just need more trees to get a good end model. 

**Coming up with predictions** [1:04:30](https://youtu.be/blyXCk4sgEg?t=1h4m30s)



In [0]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean([preds[:,0]]),y_valid[0]

In [0]:
preds.shape

Each tree is stored in an attribute called estimators_. For each tree, we will call predict in our validation set. np.stack concatenates them together on a new axis, so the resulting preds has the shape of (10, 12000) (10 trees, 12000 validation set). The mean of 10 predictions for the first data is 9.17 while the actual value is 9.10. As you can see, none of the individual prediction is close 9.10 but the mean ends up pretty good. 

In [0]:
plt.plot([metrics.r2_score(y_valid,np.mean(preds[:i+1],axis=0)) for i in range(10)])

Here is a plot of $R^2$ values given the first i trees. As we add more trees, $R^2$ improves. But it seems as though it has flattened out.

In [0]:
m = RandomForestRegressor(n_estimators=20, n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

In [0]:
m =RandomForestRegressor(n_estimators=40,n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

In [0]:
m = RandomForestRegressor(n_estimators=80,n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

As you see, adding more trees do not help much. It will not get worse but it will stop improving things much. It will not get worse but it will stop improving things much. **This is the first hyperparameter to learn to set -- a number of estimators** . A method of setting is, as many as you have time to fit and that acutally seems to be helping.

Adding more trees slows it down, but with less trees you can still get the same insights. So when Jeremy builds most of his models, he starts with 20 or 30 trees and at the end of the project or at the end of the day's work, he will use 1000 trees and run it overnight.

# Out-of-bag (OOB) score 
[1:10:04](https://youtu.be/blyXCk4sgEg?t=1h10m4s)

Sometimes your dataset will be small and you will not want to pull out a validation set because doing so means you you now do not have enough data to build a good model. However, random forests have a very clever trick called out-of-bag (OOB) error which can handle this (and more!)

What we could do is to recognize that in our first tree, some of the rows did not get used for training. What we could do is to pass those unused rows through the first tree and treat it as a validation set. For the second tree and so on. Effectively, we would have a different validation set for each tree. To calculate our prediction, we would average all the trees where that row is not used for training. If you have hundreds of trees, it is very likely that all of the rows are going to appear many times in these out of bags samples. You can then calculate RMSE, $R^2$, etc on these out-of-bag predictions.

In [0]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1,
                         oob_score=True)
m.fit(X_train,y_train)
print_score(m)

Setting oob_score to true will do exactly this and create an attribute called oob_score_ to the model and as you see in the print_score function, if it has this attributes, it will print it out at the end , 0.85.

**Question**: Wouldn't **oob_score_** always lower than the one of the entire forest [1:12:51]? The accuracy tends to be lower because each row appears in less trees in the OOB samples than it does in the full set of trees. So OOB $R^2$ will slightly underestimate how generalizable the model is, but the more trees you add, the less serious that underestimation is. 

OOB score will come in handy when setting hyper parameters [1:13:47](https://youtu.be/blyXCk4sgEg?t=1h13m47s). There will be quite a few hyperparameters that we are going to set and we would like find some automated say to set them. One way to do that is to do **grid search** . Scikit-learn has a function called grid search and you pass in a list of all the hyper parameters you want to try. It will run your model on every possible combination of all these hyperparameters you want to try. It will run your model on every possible combination of these hyperparameters and tell you which one is the best. **OOB score is a great choice for getting it to tell you which one is the best. **



# Reducing Overfitting 

**Subsampling** [1:14:52](https://youtu.be/blyXCk4sgEg?t=1h14m52s)

Earlier, we took 30,000 rows and created all the models which used a subset of 30,000 each time. Why not take a totally different subset of 30,000 each time? In other words, let's leave the entire 389,125 records as is, and we want to make thing faster, pick a different subset of 30,000 each time. So rather than bootstrapping the entire set of rows , just randomly sample a subset of the data.

In [0]:
df_trn , y_trn,nas = proc_df(df_raw,'SalePrice')
X_train,X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn,n_trn)

In [0]:
set_rf_samples(20000)

**set_rf_samples:** Just as before, we use 20,000 of them in our training set (before it was out of 30,000, this time it is out of 389,125).

In [0]:
m = RandomForestRegressor(n_jobs=-1,oob_score=True)
%time m.fit(X_train,y_train)
print_score(m)

Since each additional tree allows the model to see more data, this approach can make additional trees more useful.

In [0]:
m = RandomForestRegressor(n_estimators=40,n_jobs=-1,oob_score=True)
m.fit(X_train,y_train)
print_score(m)

This will take the same amount of time to run as before, but every tree has an access to the entire dataset. After using 40 estimators, we get the $R^2$ score of 0.87

Question: What samples is this OOB score calculated on [1:18:26](https://youtu.be/blyXCk4sgEg?t=1h18m26s)
- Scikit-learn does not support this out of box, so **set_rf_samples** is a custom function. So OOB score needs to be turned off when using **set_rf_samples** as they are not compatible. **reset_rf_samples()** will turn it back to the way it was.

**The biggest tip:** [1:20:30](https://youtu.be/blyXCk4sgEg?t=1h20m30s): Most people run all of their models on all of the data all of the time using their best possible parameters which is just pointless. If you  are trying to find out which feature is important and how they are related to each other, having the 4th decimal place of accuracy is not going to change any of your insights at all. Do most of your models on a large enough sample size that your accuracy is reasonable ( within a reasonable distance of the best accuracy you can get) and taking a small number of seconds to train so that you can interactively do your analysis.


#A couple more parameters [1:21:18](https://youtu.be/blyXCk4sgEg?t=1h21m18s)

Let's get a baseline for this full set to compare to



In [0]:
reset_rf_samples()


In [0]:
m = RandomForestRegressor(n_estimators=40,n_jobs=-1,oob_score=True)
m.fit(X_train,y_train)
print_score(m)

Here OOB is higher than validation set. This is because our validation set is a different time period whereas OOB samples are random. It is much harder to predict a different time period.

**min_sample**

In [0]:
m = RandomForestRegressor(n_estimators = 40, min_samples_leaf=3,n_jobs=-1,oob_score=True)
m.fit(X_train,y_train)
print_score(m)

- **min_samples_leaf =3** : Stop training the tree further when a leaf node has 3 or less samples (before we were going all the way down to 1). This means there will be one or two less levels of decision being made which means there are half the number of actual decision criteria we have to train (i.e faster training time).
- For each tree, rather than just taking one point, we are taking the average of at least three points what would expect the each tree to generalize better. But each tree is going to be slightly less powerful on its own.
- The numbers that work well are 1,3,5,10,25 but it is relative to your overall dataset size.
- By using 3 instead of 1, validation $R^2$ improved from 0.89 to 0.90.

**max_features** [1:24:07](https://youtu.be/blyXCk4sgEg?t=1h24m7s)

We can also increase the amount of variation amongst the trees by not only use a sample of rows for each tree, but to also using a sample of columns for each split. We do this by specifying **max_features**, which is the proportion of features to randomly select from at each split.
- None
- 0.5
- 'sqrt'
- 1,3,5,10,20,100



In [0]:
m = RandomForestRegressor(n_estimators=40,min_samples_leaf=3,max_features=0.5,n_jobs=-1,oob_score=True)
m.fit(X_train,y_train)
print_score(m)

- **max_features = 0.5:** This idea is that the less correlated your trees are with each other, the better. Imagine you had one column that was so much better than all of the other columns of being predictive that every single tree you built always started with that column. But there might be some interaction of variables where that interaction is more important than the individual column. SO if every tree always splits on the same thing the first time, you will not get much variation in those trees.
- In addition to taking a subset of rows, at every single split point, take a different subset of columns.
- **For row sampling, each new tree is based on a random set of rows, for column sampling, every individual binary split, we choose from a different subset of columns.**
- 0.5 means randomly choose a half of them. There are special values you can use such as sqrt or log2
- Good values to use are 1, 0.5, log2, or sqrt
The RMSLE of 0.22808 would get us to the top 20 of this competition - with brainless random forest with some brainless minor hyper parameter tuning. This is why Random Forest is such an important no just first step bu often only step of machine learning. It is hard to screw up.

The sklearn docs show an example of different max_features methods with increasing numbers of trees - as you see, using a subset of features on each split requires using more trees, but results in better models:

![alt text](https://camo.githubusercontent.com/c9bf9157ef5dbe0b0ea9ad4cbb87a5e42ca03c18/687474703a2f2f7363696b69742d6c6561726e2e6f72672f737461626c652f5f696d616765732f737068785f676c725f706c6f745f656e73656d626c655f6f6f625f3030312e706e67)


#Why Random Forest works so well [1:30:21](https://youtu.be/blyXCk4sgEg?t=1h30m21s)

Lets take a look at one of the split point in the small single tree.

**fitProductClassDesc** <= 7.5 will split

![alt text](https://cdn-images-1.medium.com/max/600/1*H5bgjO2doTO74XvuavDyAQ.png)

In [0]:
df_raw.fiProductClassDesc.cat.categories

Why does this even work? Imagine the only thing that mattered was Hydraulic Excavator, Track -0.0 to 2.0 Metric Tons and nothing else mattered. It can pick out a single element by first splitting fiProductClassDesc <= 5.5 then fiProductClassDesc > 4.5. With just two splits, we can pull out a single category. Tree is infinitely flexible even with a categorical variable. If there is a particular category which has different level of price, it can gradually zoom in on those group by using multiple splits. Random forest is very easy to use and very resilent. 